<a href="https://colab.research.google.com/github/nickintj/20242R0136COSE47402/blob/main/Final%20Project/Hyper_CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms
import numpy as np

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224 for CLIP
    transforms.ToTensor(),
])

# Load dataset
train_data = CIFAR100(root='./data', train=True, download=True, transform=transform)
test_data = CIFAR100(root='./data', train=False, download=True, transform=transform)

coarse_label_mapping = np.array([ 4,  1, 14,  8,  0,  6,  7,  7, 18,  3,
                               3, 14,  9, 18,  7, 11,  3,  9,  7, 11,
                               6, 11,  5, 10,  7,  6, 13, 15,  3, 15,
                               0, 11,  1, 10, 12, 14, 16,  9, 11,  5,
                               5, 18,  8,  8, 15, 13, 14, 17, 18, 10,
                               16, 4, 17,  4,  2,  0, 17,  4, 18, 17,
                               10, 3,  2, 12, 12, 16, 12,  1,  9, 18,
                               2, 10,  0,  1, 16, 12,  9, 13, 15, 13,
                              16, 18,  2,  4,  6, 18,  5,  5,  8, 18,
                              18,  1,  2, 15,  6,  0, 17,  8, 14, 13])

train_data.targets = [coarse_label_mapping[label] for label in train_data.targets]
test_data.targets = [coarse_label_mapping[label] for label in test_data.targets]

100%|██████████| 169M/169M [00:06<00:00, 26.4MB/s]


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [2]:
import torch
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Freeze pre-trained layers if required
for name, param in model.named_parameters():
    if 'vision_model' in name:
        param.requires_grad = True


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [3]:
from torch import nn

# Modify the image tower for classification
num_classes = 19
#model.classification_head = nn.Linear(model.vision_model.config.hidden_size, num_classes)
model.vision_model.head = nn.Linear(model.vision_model.config.hidden_size, num_classes)

assert all(0 <= label < 19 for label in train_data.targets), "Train labels are out of bounds."
assert all(0 <= label < 19 for label in test_data.targets), "Test labels are out of bounds."

In [4]:
coarse_class_texts=[
    'aquatic_mammals', 'fish', 'flowers', 'food_containers',
    'fruit_and_vegetables', 'household_electrical_devices',
    'household_furniture', 'insects', 'large_carnivores',
    'large_man-made_outdoor_things', 'large_natural_outdoor_scenes',
    'large_omnivores_and_herbivores', 'medium_mammals',
    'non-insect_invertebrates', 'people', 'reptiles', 'small_mammals',
    'trees', 'vehicles'
]

In [ ]:
from torch import nn, optim
from tqdm import tqdm
from torch.utils.data import DataLoader

# Set the model to train mode
model.train()

# Define an optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-5)

# Loss function
loss_fn = nn.CrossEntropyLoss()
epochs = 5
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
# Training loop
for epoch in range(epochs):
    for images, labels in tqdm(train_loader):
        optimizer.zero_grad()

        # Process images and text
        inputs_image = processor(images=images, return_tensors="pt")
        text_inputs = processor(text=coarse_class_texts, return_tensors="pt", padding=True)

        # Forward pass through the model
        outputs = model(**inputs_image, input_ids=text_inputs.input_ids)

        # Calculate loss
        logits_per_image = outputs.logits_per_image
        logits_per_text = outputs.logits_per_text

        # Compute contrastive loss
        loss = loss_fn(logits_per_image, labels)  # or use a custom contrastive loss function

        # Backpropagation
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")


  7%|▋         | 56/782 [28:19<6:04:57, 30.16s/it]

In [ ]:
from tqdm import tqdm
model.eval()

correct = 0
total = 0
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        inputs_image = processor(images=images, return_tensors="pt")
        text_inputs = processor(text=coarse_class_texts, return_tensors="pt", padding=True)

        # Get model predictions
        outputs = model(**inputs_image, input_ids=text_inputs.input_ids)

        logits_per_image = outputs.logits_per_image
        predicted = logits_per_image.argmax(dim=-1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

In [1]:
'''
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

# Define DataLoader
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Load the CLIP model
from transformers import CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

# Optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)  # Use a smaller learning rate for fine-tuning
criterion = CrossEntropyLoss()

# Train loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(10):  # Number of epochs
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Preprocess the images using the processor (CLIP handles normalization)
        inputs = processor(images=inputs, return_tensors="pt").pixel_values.to(device)

        input_ids = torch.tensor([processor.tokenizer.encode("dummy text")]).to(device)

        # Forward pass: pass only pixel_values
        outputs = model(input_ids=input_ids, pixel_values=inputs)  # Now using the modified model
        logits_per_image = outputs.logits_per_image  # Shape: (batch_size, num_classes)

        # Compute loss
        loss = criterion(logits_per_image, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")
    '''

NameError: name 'train_data' is not defined